In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/tmp/ipykernel_42533/2080034654.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Baseline

In [2]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [3]:
train = pd.read_csv("/home/ana/University/MitigatingBiasInDataScience/processed_data/Adult_train.csv", index_col=0)


In [5]:
classifier = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)
classifier.fit(X, y_true)
y_pred = classifier.predict(X)
mf = MetricFrame(metrics=accuracy_score, y_true=y_true, y_pred=y_pred, sensitive_features=sex)
mf.overall

0.8446197991391678

In [6]:
mf.by_group

sex
Female    0.925260
Male      0.804624
Name: accuracy_score, dtype: float64

In [6]:
y_train  = train["income"]
X_train = train.drop(columns=["income"])
sex = X_train["sex"]

In [7]:
from fairlearn.reductions import DemographicParity, ExponentiatedGradient
np.random.seed(0)  # set seed for consistent results with ExponentiatedGradient

constraint = DemographicParity()
classifier = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)
mitigator = ExponentiatedGradient(classifier, constraint)
mitigator.fit(X_train, y_train, sensitive_features=sex)
y_pred_mitigated = mitigator.predict(X_train)



/home/ana/miniconda3/envs/FairEnv/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/home/ana/miniconda3/envs/FairEnv/lib/python3.

In [8]:
sr_mitigated = MetricFrame(metrics=accuracy_score, y_true=y_train, y_pred=y_pred_mitigated, sensitive_features=sex)
sr_mitigated.overall

0.8261917966771961

In [9]:
X_train

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
38339,78,1,4,2,1,9,5,1,2,0,0,20,7
23421,64,1,4,6,2,1,1,1,2,0,0,56,1
8309,46,1,1,9,1,3,2,1,1,0,0,60,1
13298,37,1,2,10,2,6,1,2,2,0,0,30,1
22944,38,1,2,11,2,9,1,1,2,0,0,50,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21677,22,1,1,9,1,6,2,1,1,0,0,40,1
28940,46,1,3,13,2,1,1,1,2,0,1848,45,1
30435,47,1,1,9,3,3,4,1,1,0,0,37,1
43425,43,1,4,2,3,10,2,1,2,0,0,28,2


In [11]:
X_train["income"]  = y_pred_mitigated

In [13]:
import sys
sys.path.insert(0, "..")
from fair_metrics.group_fairness import disparity_ratio
from fair_metrics.individual_fairness import equalized_odds

In [14]:
disparity_ratio(X_train, "income", "sex", 2, 1)

0.9110128638849009

In [15]:
equalized_odds(X_train, "income", "sex", "education", lambda x: x == 5, 2, 1)

0.6133882749578573

In [19]:
equalized_odds(X_train, "income", "sex", "marital-status", lambda x: x == 1, 2, 1)

1.0463911872115081

In [17]:
equalized_odds(X_train, "income", "sex", "hours-per-week", lambda x: x < 40, 2, 1)

1.404206905127036